In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2
from skimage.transform import rotate, AffineTransform, warp
import matplotlib.pyplot as plt
import random
from skimage import img_as_ubyte
import os
from skimage.util import random_noise

# ------------------------------------------------------------------------------------------------------------

test = pd.read_csv("/kaggle/input/test_vc2kHdQ.csv")
train = pd.read_csv("/kaggle/input/train_SOaYf6m/train.csv")

train_path = "/kaggle/input/train_SOaYf6m/images/"
test_path = "/kaggle/input/train_SOaYf6m/images/"

print(train.emergency_or_not.value_counts())

# --------------------------------------------------------------------------------------------------------------
import cv2
import os

indexes = train.index
training_data = []

size = 224

path_file = train_path

def create_training_data():
    for i in indexes:
        path = os.path.join(path_file, train.loc[i, 'image_names'])
#         print("Image No: ",i, )
        class_idx = train.loc[i, 'emergency_or_not']

        img_array = cv2.imread(path, cv2.IMREAD_COLOR)
        new_array = cv2.resize(img_array, (size, size))
        training_data.append([new_array, class_idx])

create_training_data() 

x = []
y = []

for a, b in training_data:
    x.append(a)
    y.append(b)

# del training_data

X = np.array(x).reshape(-1, size, size, 3)

# del x

# plt.imshow(X[0])
# plt.show()

# import pickle

# pickle_out = open("X.pickle","wb")
# pickle.dump(X, pickle_out)
# pickle_out.close()

# pickle_out = open("y.pickle","wb")
# pickle.dump(y, pickle_out)
# pickle_out.close()

# del X, y

# pickle_out = open("y_train.pickle","wb")
# pickle.dump(y_train, pickle_out)
# pickle_out.close()

# pickle_out = open("y_test.pickle","wb")
# pickle.dump(y_test, pickle_out)
# pickle_out.close()

# __________________________________________________________________________________________________________
indexes = test.index
test_data = []

size = 224

path_file = test_path

def create_test_data():
    for i in indexes:
        path = os.path.join(path_file, test.loc[i, 'image_names'])
        
        # class_idx = train.loc[i, 'Class']
#         print("Image No: ",i," !!!")
        img_array = cv2.imread(path, cv2.IMREAD_COLOR)
        new_array = cv2.resize(img_array, (size, size))
        test_data.append([new_array, 0])

create_test_data()

x_pred = []
y_pred = []

for a, b in test_data:
    x_pred.append(a)
    y_pred.append(b)

X_pred = np.array(x_pred).reshape(-1, size, size, 3)
print(X_pred.shape)

import pickle

# pickle_out = open("X_pred.pickle","wb")
# pickle.dump(X_pred, pickle_out)
# pickle_out.close()



In [ ]:
index = 201

img_array = X[index]
plt.imshow(img_array)
plt.show()

print(y[index])

# resnet50

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2
from skimage.transform import rotate, AffineTransform, warp
import matplotlib.pyplot as plt
import random
from skimage import img_as_ubyte
import os
from skimage.util import random_noise
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle
import time
import numpy as np

import tensorflow.keras

NCLASSES = 2
HEIGHT = 224
WIDTH = 224
NUM_CHANNELS = 3
BATCH_SIZE = 32


# ResNet152
# base_model = tf.keras.applications.VGG19(input_shape=(HEIGHT, WIDTH, NUM_CHANNELS), include_top=False, weights='imagenet')

# ResNet50
# base_model = tf.keras.applications.ResNet50(input_shape=(HEIGHT, WIDTH, NUM_CHANNELS), include_top=False, weights='imagenet')

# ResNet101
base_model = tf.keras.applications.ResNet101(input_shape=(HEIGHT, WIDTH, NUM_CHANNELS), include_top=False, weights='imagenet')

# EfficientNetB7
# base_model = tf.keras.applications.DenseNet201(input_shape=(HEIGHT, WIDTH, NUM_CHANNELS), include_top=False, weights='imagenet')


base_model.trainable = False

print(base_model.summary())

In [ ]:
X.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split as tts
import pickle
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=2)

tf.compat.v2.random.set_seed(1)

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
# x = layers.Dense(1024, activation='relu')(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(256, activation='relu')(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(128, activation='relu')(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(128, activation='relu')(x)
# x = layers.Dropout(0.5)(x)
x = layers.Dense(2, activation='softmax')(x)

model_3 = models.Model(inputs=base_model.input, outputs=x)
# print(model_3.summary())

# X = np.array(pickle.load(open("X.pickle", "rb")))
# y = np.array(pickle.load(open("y.pickle", "rb")))

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# model_3.fit_generator(datagen.flow(X_train, y_train, batch_size=32),steps_per_epoch=len(X_train) / 32, epochs=34, 
#                       validation_data = [np.array(X_test), np.array(y_test)])

X_train, X_test, y_train, y_test = tts(X, y, random_state = 21, test_size=0.1)

from tensorflow.keras import utils

# num_classes = 2
# y_train = utils.to_categorical(y_train, num_classes)
# y_test = utils.to_categorical(y_test, num_classes)

datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True)

# train_gen = datagen.fit(X_train, augment=True, seed=1)

model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# resnet50
model_3.fit(np.array(X_train), np.array(y_train), epochs = 50, validation_data = [np.array(X_test), np.array(y_test)])
# 35

# model_3.fit_generator(datagen.flow(X_train, y_train, batch_size=32), epochs=37,, validation_data = [np.array(X_test), np.array(y_test)])

In [ ]:
pred = model_3.predict([X_pred])
pred_class = np.argmax(pred, axis = 1)
pred_class.shape
temp = pd.DataFrame(pred_class)
temp.to_csv("resnet101_train_layers_1024_48.csv")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split as tts
import pickle
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=2)

tf.compat.v2.random.set_seed(1)

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
# x = layers.Dense(256, activation='relu')(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(128, activation='relu')(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(128, activation='relu')(x)
# x = layers.Dropout(0.5)(x)
x = layers.Dense(2, activation='softmax')(x)

model_3 = models.Model(inputs=base_model.input, outputs=x)
# print(model_3.summary())

# X = np.array(pickle.load(open("X.pickle", "rb")))
# y = np.array(pickle.load(open("y.pickle", "rb")))

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# model_3.fit_generator(datagen.flow(X_train, y_train, batch_size=32),steps_per_epoch=len(X_train) / 32, epochs=34, 
#                       validation_data = [np.array(X_test), np.array(y_test)])

X_train, X_test, y_train, y_test = tts(X, y, random_state = 21, test_size=0.1)

from tensorflow.keras import utils

# num_classes = 2
# y_train = utils.to_categorical(y_train, num_classes)
# y_test = utils.to_categorical(y_test, num_classes)

datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True)

# train_gen = datagen.fit(X_train, augment=True, seed=1)

model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# resnet50
model_3.fit(np.array(X_train), np.array(y_train), epochs = 29, validation_data = [np.array(X_test), np.array(y_test)])
# 13, 15, 29, 30, 41, 49


# model_3.fit_generator(datagen.flow(X_train, y_train, batch_size=32), epochs=37,, validation_data = [np.array(X_test), np.array(y_test)])


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split as tts
import pickle
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=5)

tf.compat.v2.random.set_seed(123)

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(2, activation='softmax')(x)

model_3 = models.Model(inputs=base_model.input, outputs=x)
# print(model_3.summary())

# X = np.array(pickle.load(open("X.pickle", "rb")))
# y = np.array(pickle.load(open("y.pickle", "rb")))

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# model_3.fit_generator(datagen.flow(X_train, y_train, batch_size=32),steps_per_epoch=len(X_train) / 32, epochs=34, 
#                       validation_data = [np.array(X_test), np.array(y_test)])

X_train, X_test, y_train, y_test = tts(X, y, random_state = 21, test_size=0.1)

from tensorflow.keras import utils

# num_classes = 2
# y_train = utils.to_categorical(y_train, num_classes)
# y_test = utils.to_categorical(y_test, num_classes)

# datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=5,
#     width_shift_range=0.05,
#     height_shift_range=0.05,
#     horizontal_flip=True)

# datagen.fit(X_train, augment=True, seed=1)

model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_3.fit(np.array(X_train), np.array(y_train), epochs = 31, validation_data = [np.array(X_test), np.array(y_test)])

# model_3.fit_generator(datagen.flow(X_train, y_train, batch_size=32), epochs=56, validation_data = [np.array(X_test), np.array(y_test)])


In [ ]:
pred = model_3.predict([X_pred])
pred_class = np.argmax(pred, axis = 1)
pred_class.shape
temp = pd.DataFrame(pred_class)
temp.to_csv("resnet101_train_layers_1024_epochs_31.csv")

In [ ]:
tf.compat.v2.random.set_seed(1)
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
# x = layers.Dense(128, activation='relu')(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(128, activation='relu')(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(128, activation='relu')(x)
# x = layers.Dropout(0.5)(x)
# # x = layers.Dense(128, activation='relu')(x)
# # x = layers.Dropout(0.5)(x)
x = layers.Dense(2, activation='softmax')(x)

model_f = models.Model(inputs=base_model.input, outputs=x)
model_f.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_f.fit(np.array(X), np.array(y), epochs=19)

# Part 1b: XGB

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split as tts
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X_res = X.reshape(X.shape[0], X.shape[1]*X.shape[2]*X.shape[3])

x = StandardScaler().fit_transform(X_res)
print(x.shape)

pca = PCA(n_components=100)
x_pca = pca.fit_transform(x)
print(pca.explained_variance_ratio_.sum())

In [ ]:
X_train, X_test, y_train, y_test = tts(x_pca, y, random_state = 21)

model = XGBClassifier(tree_method = 'gpu_hist')
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
from tensorflow.keras import utils

num_classes = 2
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [ ]:
y_train.shape

In [ ]:
pred = model_3.predict([X_pred])
pred_class = np.argmax(pred, axis = 1)
pred_class.shape
temp = pd.DataFrame(pred_class)
temp.to_csv("vgg16_layers_5_epochs_34.csv")

# Part 2: Augmentation Try

In [ ]:
def convert(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
    return image, label

def augment(image,label):
    image,label = convert(image, label)
    image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
    image = tf.image.resize_with_crop_or_pad(image, 34, 34) # Add 6 pixels of padding
    image = tf.image.random_crop(image, size=[28, 28, 1]) # Random crop back to 28x28
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
    return image,label

In [ ]:
import tensorflow as tf

In [ ]:
augmented_train_batches = (
    train_dataset
    # Only train on a subset, so you can quickly see the effect.
    .take(NUM_EXAMPLES)
    .cache()
    .shuffle(num_train_examples//4)
    # The augmentation is added here.
    .map(augment, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
) 

In [ ]:
model_with_aug = make_model()

aug_history = model_with_aug.fit(augmented_train_batches, epochs=50, validation_data=validation_batches)